In [27]:
import numpy as np
import pandas as pd
import os
import nibabel as nib
from scipy import ndimage
import torch
from torch.utils.data import Dataset, DataLoader

In [28]:
# Exploratory Data Analysis
# Realised that we have much more MCI Data, hence decided to move ahead with three category classification
df = pd.read_csv("/kaggle/input/adni-processed/ADNI1_Complete_1Yr_1.5T_6_20_2025.csv")
print("Dataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())
print("\nLabel distribution:")
print(df['Group'].value_counts())
print(f"\nUnique subjects: {df['Subject'].nunique()}")
print(f"Total scans: {len(df)}")

Dataset shape: (2294, 12)

Columns: ['Image Data ID', 'Subject', 'Group', 'Sex', 'Age', 'Visit', 'Modality', 'Description', 'Type', 'Acq Date', 'Format', 'Downloaded']

First few rows:
  Image Data ID     Subject Group Sex  Age Visit Modality  \
0       I112538  941_S_1311   MCI   M   70   m12      MRI   
1        I97341  941_S_1311   MCI   M   70   m06      MRI   
2        I97327  941_S_1311   MCI   M   69    sc      MRI   
3        I75150  941_S_1202    CN   M   78   m06      MRI   
4       I105437  941_S_1202    CN   M   79   m12      MRI   

                                  Description       Type   Acq Date Format  \
0    MPR; GradWarp; B1 Correction; N3; Scaled  Processed  6/01/2008  NiFTI   
1  MPR-R; GradWarp; B1 Correction; N3; Scaled  Processed  9/27/2007  NiFTI   
2    MPR; GradWarp; B1 Correction; N3; Scaled  Processed  3/02/2007  NiFTI   
3    MPR; GradWarp; B1 Correction; N3; Scaled  Processed  8/24/2007  NiFTI   
4    MPR; GradWarp; B1 Correction; N3; Scaled  Processed  

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [29]:
# Understanding how the directory is structured
base_dir = "/kaggle/input/adni-processed/ADNI1_Processed/ADNI1_Processed/"
def explore_directory_structure(base_path, max_depth=3, current_depth=0):
    items = []
    if current_depth >= max_depth:
        return items
    for item in os.listdir(base_path)[:10]:  
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            print("  " * current_depth + f"{item}/")
            items.extend(explore_directory_structure(item_path, max_depth, current_depth + 1))
        else:
            print("  " * current_depth + f"{item}")
            if item.endswith(('.nii', '.nii.gz')):
                items.append(item_path)
    return items
nii_files = explore_directory_structure(base_dir)
print(f"\nFound {len(nii_files)} .nii files in sample")


133_S_0913/
  I92637/
    ADNI_133_S_0913_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080225185003300_S35319_I92637_mni_norm.nii
  I119636/
    ADNI_133_S_0913_MR_MPR__GradWarp__B1_Correction__N3__Scaled_2_Br_20081008095615528_S35319_I119636_mni_norm.nii
094_S_1090/
  I63176/
    ADNI_094_S_1090_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070731120023063_S23375_I63176_mni_norm.nii
941_S_1194/
  I75141/
    ADNI_941_S_1194_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070925114520658_S38206_I75141_mni_norm.nii
031_S_0618/
  I91766/
    ADNI_031_S_0618_MR_MPR-R__GradWarp__N3__Scaled_Br_20080223132226600_S24408_I91766_mni_norm.nii
073_S_0312/
  I39881/
    ADNI_073_S_0312_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070213204740079_S15079_I39881_mni_norm.nii
137_S_0669/
  I66241/
    ADNI_137_S_0669_MR_MPR__GradWarp__N3__Scaled_Br_20070808215314178_S27107_I66241_mni_norm.nii
062_S_0535/
  I50426/
    ADNI_062_S_0535_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_200704241

In [30]:
# Mapping the MRI Scans present in the ADNI_Processed directory to the respective metadata in the CSV file
# Note that there has been limited mapping as the CSV file correponds to the entire ADNI Dataset, but I am working on a 10 gb subset
def simple_file_mapping(df, base_dir):
    mapped_data = []
    
    for idx, row in df.iterrows():
        subject_id = row['Subject']
        image_id = row['Image Data ID']
        group = row['Group']
        
        # Build path to subject directory
        subject_dir = os.path.join(base_dir, subject_id)
        
        # Check if subject directory exists
        if os.path.exists(subject_dir):
            # Look through all subdirectories in the subject folder
            for subfolder in os.listdir(subject_dir):
                subfolder_path = os.path.join(subject_dir, subfolder)
                
                # Check if this subfolder matches our image_id or contains it
                if os.path.isdir(subfolder_path):
                    # Check if image_id matches subfolder name exactly
                    if subfolder == image_id:
                        nii_files = [f for f in os.listdir(subfolder_path) if f.endswith('.nii')]
                        if nii_files:
                            file_path = os.path.join(subfolder_path, nii_files[0])
                            mapped_data.append({
                                'subject_id': subject_id,
                                'image_id': image_id,
                                'group': group,
                                'file_path': file_path
                            })
                            break  # Found the file, move to next CSV row
                    
                    # Also check if image_id is contained in the subfolder name
                    elif image_id in subfolder:
                        nii_files = [f for f in os.listdir(subfolder_path) if f.endswith('.nii')]
                        if nii_files:
                            file_path = os.path.join(subfolder_path, nii_files[0])
                            mapped_data.append({
                                'subject_id': subject_id,
                                'image_id': image_id,
                                'group': group,
                                'file_path': file_path
                            })
                            break  # Found the file, move to next CSV row
    
    return pd.DataFrame(mapped_data)

base_dir = "/kaggle/input/adni-processed/ADNI1_Processed/ADNI1_Processed"
mapped_df = simple_file_mapping(df, base_dir)
print(f"Successfully mapped {len(mapped_df)} files")
print("\nSample mappings:")
for i in range(min(3, len(mapped_df))):
    row = mapped_df.iloc[i]
    print(f"Subject: {row['subject_id']} | Image ID: {row['image_id']} | Label: {row['group']}")
    print(f"File: {row['file_path']}")
    print()

def check_scan_exists(image_id, mapped_df):
    exists = image_id in mapped_df['image_id'].values
    if exists:
        scan_info = mapped_df[mapped_df['image_id'] == image_id].iloc[0]
        print(f"Found: {image_id} - Subject: {scan_info['subject_id']}, Label: {scan_info['group']}")
        print(f"Path: {scan_info['file_path']}")
    else:
        print(f"Not found: {image_id}")
    return exists

# Checking if a random MRI scan has been mapped
check_scan_exists('I68056', mapped_df)


Successfully mapped 459 files

Sample mappings:
Subject: 941_S_1311 | Image ID: I112538 | Label: MCI
File: /kaggle/input/adni-processed/ADNI1_Processed/ADNI1_Processed/941_S_1311/I112538/ADNI_941_S_1311_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080703170241434_S51039_I112538_mni_norm.nii

Subject: 941_S_1311 | Image ID: I97327 | Label: MCI
File: /kaggle/input/adni-processed/ADNI1_Processed/ADNI1_Processed/941_S_1311/I97327/ADNI_941_S_1311_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080313130949784_S27408_I97327_mni_norm.nii

Subject: 941_S_1197 | Image ID: I66462 | Label: CN
File: /kaggle/input/adni-processed/ADNI1_Processed/ADNI1_Processed/941_S_1197/I66462/ADNI_941_S_1197_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070809164131486_S25332_I66462_mni_norm.nii

Found: I68056 - Subject: 021_S_0626, Label: MCI
Path: /kaggle/input/adni-processed/ADNI1_Processed/ADNI1_Processed/021_S_0626/I68056/ADNI_021_S_0626_MR_MPR-R__GradWarp__N3__Scaled_Br_20070816132730116_S26663_I680

True

In [31]:
# Understanding data distribution in my ADNI subset
filtered_df = mapped_df.copy()
filtered_df['label'] = filtered_df['group'].map({'CN': 0, 'MCI': 1, 'AD': 2})

print(f"Three-class dataset: {len(filtered_df)} samples")
print("Distribution:")
print(filtered_df['group'].value_counts())
print("\nPercentages:")
for group, count in filtered_df['group'].value_counts().items():
    print(f"{group}: {count/len(filtered_df)*100:.1f}%")



Three-class dataset: 459 samples
Distribution:
group
MCI    225
CN     139
AD      95
Name: count, dtype: int64

Percentages:
MCI: 49.0%
CN: 30.3%
AD: 20.7%


In [32]:
# Moving ahead with the train-test split based on unique subjects
# I noted after some exploration that one medical subject might have multiple MRI scans, hence I ensured that all scans of one person are in the same set
from sklearn.model_selection import train_test_split
unique_subjects = filtered_df['subject_id'].unique()
subject_labels = filtered_df.groupby('subject_id')['group'].first()
print(f"Unique subjects: {len(unique_subjects)}")
train_subjects, test_subjects = train_test_split(
    unique_subjects,
    test_size=0.2, 
    random_state=3,
    stratify=subject_labels
)

train_subjects_list = list(train_subjects)
subject_labels_train = subject_labels[train_subjects_list]

final_train_subjects, val_subjects = train_test_split(
    train_subjects_list,
    test_size=0.25,
    random_state=69,
    stratify=subject_labels_train
)

train_df = filtered_df[filtered_df['subject_id'].isin(final_train_subjects)].copy()
val_df = filtered_df[filtered_df['subject_id'].isin(val_subjects)].copy()
test_df = filtered_df[filtered_df['subject_id'].isin(test_subjects)].copy()

print(f"Training samples: {len(train_df)}")
print("Training distribution:")
print(train_df['group'].value_counts())

print(f"\nValidation samples: {len(val_df)}")
print("Validation distribution:")
print(val_df['group'].value_counts())

print(f"\nTest samples: {len(test_df)}")
print("Test distribution:")
print(test_df['group'].value_counts())



Unique subjects: 355
Training samples: 277
Training distribution:
group
MCI    136
CN      87
AD      54
Name: count, dtype: int64

Validation samples: 97
Validation distribution:
group
MCI    47
CN     29
AD     21
Name: count, dtype: int64

Test samples: 85
Test distribution:
group
MCI    42
CN     23
AD     20
Name: count, dtype: int64


In [33]:
class ADNIDataset(Dataset):
    def __init__(self, df, target_shape=(128, 128, 128)):
        self.df = df.reset_index(drop=True)
        self.target_shape = target_shape
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load MRI scan
        img = nib.load(row['file_path'])
        volume = img.get_fdata()
        
        # Preprocess following medical imaging best practices
        volume = self.preprocess_volume(volume)
        volume_tensor = torch.FloatTensor(volume).unsqueeze(0)  # Add channel dimension
        
        label = row['label']
        return volume_tensor, label
    
    def preprocess_volume(self, volume):
        # Intensity normalization (zero mean, unit variance)
        volume = (volume - volume.mean()) / (volume.std() + 1e-8)
        
        # Clip extreme values
        volume = np.clip(volume, -3, 3)
        
        # Resize to target shape for memory efficiency
        if volume.shape != self.target_shape:
            zoom_factors = [self.target_shape[i] / volume.shape[i] for i in range(3)]
            volume = ndimage.zoom(volume, zoom_factors, order=1)
        
        return volume.astype(np.float32)

# Create datasets
train_dataset = ADNIDataset(train_df)
val_dataset = ADNIDataset(val_df)
test_dataset = ADNIDataset(test_df)

# Create data loaders with small batch sizes for 3D volumes
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=2)


In [34]:
# Loading and exploring the npz file with adnet weights
npz_path = "/kaggle/input/adnet-weights/adnet.npz"
data = np.load(npz_path, allow_pickle=True)

print("Keys in the .npz file:")
print(data.files)

print("\nDetailed information:")
for key in data.files:
    array = data[key]
    print(f"{key}: shape={array.shape}, dtype={array.dtype}")
    print(f"  Value range: [{array.min():.6f}, {array.max():.6f}]")
    print(f"  Mean: {array.mean():.6f}, Std: {array.std():.6f}")
    print()

conv_weights = []
conv_biases = []
linear_weights = []
linear_biases = []

for key in data.files:
    shape = data[key].shape
    if len(shape) == 5:  # Conv3D weights
        conv_weights.append((key, shape))
    elif len(shape) == 1:  # Biases
        if shape[0] in [32, 64, 128, 256]:  # Conv biases
            conv_biases.append((key, shape))
        else:  # Linear biases
            linear_biases.append((key, shape))
    elif len(shape) == 2:  # Linear weights
        linear_weights.append((key, shape))

print(f"Conv3D weights: {len(conv_weights)}")
print(f"Conv3D biases: {len(conv_biases)}")
print(f"Linear weights: {len(linear_weights)}")
print(f"Linear biases: {len(linear_biases)}")


Keys in the .npz file:
['arr_24', 'arr_25', 'arr_26', 'arr_27', 'arr_20', 'arr_21', 'arr_22', 'arr_23', 'arr_28', 'arr_29', 'arr_46', 'arr_47', 'arr_44', 'arr_45', 'arr_42', 'arr_43', 'arr_40', 'arr_41', 'arr_48', 'arr_49', 'arr_54', 'arr_33', 'arr_32', 'arr_31', 'arr_30', 'arr_37', 'arr_36', 'arr_35', 'arr_34', 'arr_39', 'arr_38', 'arr_19', 'arr_18', 'arr_51', 'arr_50', 'arr_53', 'arr_52', 'arr_11', 'arr_10', 'arr_13', 'arr_12', 'arr_15', 'arr_14', 'arr_17', 'arr_16', 'arr_1', 'arr_0', 'arr_3', 'arr_2', 'arr_5', 'arr_4', 'arr_7', 'arr_6', 'arr_9', 'arr_8']

Detailed information:
arr_24: shape=(256,), dtype=float32
  Value range: [0.080202, 0.262068]
  Mean: 0.154666, Std: 0.036586

arr_25: shape=(256, 256, 3, 3, 3), dtype=float32
  Value range: [-0.064250, 0.068091]
  Mean: -0.002131, Std: 0.010475

arr_26: shape=(256,), dtype=float32
  Value range: [-0.056514, 0.071657]
  Mean: 0.005688, Std: 0.023362

arr_27: shape=(256,), dtype=float32
  Value range: [0.707229, 1.003647]
  Mean: 0.

In [35]:
# Will bring thi into use later if accuracy and other results are not satisfactory

'''
mean_std_data = np.load("/kaggle/input/adnet-normal/mean_std.npz")

print("Files in mean_std.npz:")
print(mean_std_data.files)

# Load mean and std
mean = mean_std_data['mean']
std = mean_std_data['std']

print(f"Mean: {mean}")
print(f"Std: {std}")
print(f"Mean shape: {mean.shape}")
print(f"Std shape: {std.shape}")
'''


'\nmean_std_data = np.load("/kaggle/input/adnet-normal/mean_std.npz")\n\nprint("Files in mean_std.npz:")\nprint(mean_std_data.files)\n\n# Load mean and std\nmean = mean_std_data[\'mean\']\nstd = mean_std_data[\'std\']\n\nprint(f"Mean: {mean}")\nprint(f"Std: {std}")\nprint(f"Mean shape: {mean.shape}")\nprint(f"Std shape: {std.shape}")\n'

In [36]:
import torch
import torch.nn as nn
import numpy as np

class ADNet3D(nn.Module):
    def __init__(self, num_classes=3):
        super(ADNet3D, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2, stride=2),
            
            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2, stride=2),
            
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.Conv3d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2, stride=2),
            
            nn.Conv3d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2, stride=2),
            
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2, stride=2),
        )

        self.adaptive_pool = nn.AdaptiveAvgPool3d((2, 2, 2))
        
        self.classifier = nn.Sequential(
            nn.Linear(16384, 512),
            nn.LayerNorm(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            
            nn.Linear(512, 512),
            nn.LayerNorm(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            
            nn.Linear(512, num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
'''
def load_adnet_weights(model, npz_path):
    data = np.load(npz_path, allow_pickle=True)
    
    # Get all .npz arrays sorted by name
    npz_arrays = sorted([k for k in data.keys() if k.startswith('arr_')])
    
    # Get model parameters in order
    model_params = list(model.named_parameters())
    
    # Match .npz arrays to model parameters by shape
    mapping = {}
    used_arrays = set()
    
    for param_name, param in model_params:
        param_shape = tuple(param.shape)
        
        # Find .npz array with exact matching shape
        for arr_key in npz_arrays:
            if arr_key in used_arrays:
                continue
            
            arr_shape = tuple(data[arr_key].shape)
            
            # Create mapping when shapes match exactly
            if arr_shape == param_shape:
                mapping[arr_key] = param_name
                used_arrays.add(arr_key)
                print(f"Mapped: {arr_key} → {param_name} (shape: {param_shape})")
                break
    
    # Load weights into model state_dict
    state_dict = {}
    for arr_key, param_name in mapping.items():
        weight_tensor = torch.from_numpy(data[arr_key]).float()
        state_dict[param_name] = weight_tensor
    
    missing_keys, unexpected_keys = model.load_state_dict(state_dict, strict=False)
    
    print(f"Successfully mapped: {len(mapping)} parameters")
    print(f"Missing keys: {len(missing_keys)}")
    
    return model, len(mapping) > 0
'''
model = ADNet3D(num_classes=3)
# model, loading_success = load_adnet_weights(model, "/kaggle/input/adnet-weights/adnet.npz")
# print(f"Model created with {sum(p.numel() for p in model.parameters()):,} parameters")


In [37]:
print(type(model)) 
print(hasattr(model, 'parameters'))  
print(train_df.columns)

<class '__main__.ADNet3D'>
True
Index(['subject_id', 'image_id', 'group', 'file_path', 'label'], dtype='object')


In [38]:
device = torch.device('cuda') 
model = torch.nn.DataParallel(model, device_ids=[0, 1]).to(device)

'''
for param in model.module.features.parameters():
    param.requires_grad = False
for param in model.module.classifier.parameters():
    param.requires_grad = True
'''

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_one_epoch(epoch_index):
    running_loss = 0
    running_corrects = 0
    total_samples = 0
    epoch_corrects = 0
    epoch_samples = 0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += labels.size(0)
        
        epoch_corrects += torch.sum(preds == labels.data)
        epoch_samples += labels.size(0)
        
        if(i%50==49):
            last_loss = running_loss/50
            batch_acc = running_corrects.double() / total_samples
            print('  batch {} loss: {:.4f} acc: {:.4f}'.format(i + 1, last_loss, batch_acc))
            running_loss = 0
            running_corrects = 0
            total_samples = 0

    return last_loss, epoch_corrects.double() / epoch_samples

In [ ]:
EPOCHS = 100
best_vloss = 1000000000
best_vacc = 0.0

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))
    model.train()
    avg_loss, avg_acc = train_one_epoch(epoch)
    
    running_vloss = 0.0
    running_vcorrects = 0
    total_vsamples = 0
    
    model.eval()
    with torch.no_grad():
        for i, vdata in enumerate(val_loader):
            vinputs, vlabels = vdata
            vinputs, vlabels = vinputs.to(device), vlabels.to(device)
            voutputs = model(vinputs)
            vloss = criterion(voutputs, vlabels)
            running_vloss += vloss.item()
            
            _, vpreds = torch.max(voutputs, 1)
            running_vcorrects += torch.sum(vpreds == vlabels.data)
            total_vsamples += vlabels.size(0)

    avg_vloss = running_vloss / len(val_loader)
    avg_vacc = running_vcorrects.double() / total_vsamples
    print('LOSS train {:.4f} valid {:.4f} ACC train {:.4f} valid {:.4f}'.format(avg_loss, avg_vloss, avg_acc, avg_vacc))
    if avg_vacc > best_vacc:
        best_vacc = avg_vacc


EPOCH 1:
  batch 50 loss: 1.2744 acc: 0.4900
  batch 100 loss: 1.1906 acc: 0.3400
LOSS train 1.1906 valid 1.2014 ACC train 0.4477 valid 0.2990
EPOCH 2:
  batch 50 loss: 1.0849 acc: 0.4800
  batch 100 loss: 1.1040 acc: 0.4400
LOSS train 1.1040 valid 1.0913 ACC train 0.4513 valid 0.2990
EPOCH 3:
  batch 50 loss: 1.1741 acc: 0.3200
  batch 100 loss: 1.0829 acc: 0.4400
LOSS train 1.0829 valid 1.0628 ACC train 0.3971 valid 0.4845
EPOCH 4:
  batch 50 loss: 1.0453 acc: 0.4700
  batch 100 loss: 1.0978 acc: 0.4400
LOSS train 1.0978 valid 1.0493 ACC train 0.4404 valid 0.4845
EPOCH 5:
  batch 50 loss: 1.0308 acc: 0.4800
  batch 100 loss: 1.2190 acc: 0.4100
LOSS train 1.2190 valid 1.0587 ACC train 0.4296 valid 0.4845
EPOCH 6:
  batch 50 loss: 1.1262 acc: 0.4600
  batch 100 loss: 1.0967 acc: 0.4700
LOSS train 1.0967 valid 1.1340 ACC train 0.4765 valid 0.4845
EPOCH 7:
  batch 50 loss: 1.1737 acc: 0.3700
  batch 100 loss: 1.1183 acc: 0.4300
LOSS train 1.1183 valid 1.0646 ACC train 0.4404 valid 0.4845